<a href="https://colab.research.google.com/github/rikharigaurav/FSDL-text-recognizer/blob/main/Pytorch_lighting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pytorch Lighting



##Setup

In [3]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 48.6 MB/s eta 0:00:00


In [ ]:
!pip install -U 'torch_xla>=1.13'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.1
    Uninstalling google-api-core-2.11.1:
      Successfully uninstalled google-api-core-2.11.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency con

In [4]:
import torch

In [5]:
lab_idx = 2

if "bootstrap" not in locals() or bootstrap.run:
    # path management for Python
    pythonpath, = !echo $PYTHONPATH
    if "." not in pythonpath.split(":"):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    # change into the lab directory
    bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow "hot-reloading" of modules
    %load_ext autoreload
    %autoreload 2
    # needed for inline plots in some contexts
    %matplotlib inline

    bootstrap.run = False  # change to True re-run setup

!pwd
%ls

env: PYTHONPATH=.:/env/python
.:/env/python
/content/fsdl-text-recognizer-2022-labs/lab02
notebooks/  text_recognizer/  training/


#Why Lighting

PyTorch is a powerful library for executing differentiable
tensor operations with hardware acceleration
and it includes many neural network primitives,
but it has no concept of "training".
At a high level, an `nn.Module` is a stateful function with gradients
and a `torch.optim.Optimizer` can update that state using gradients,
but there's no pre-built tools in PyTorch to iteratively generate those gradients from data.

So the first thing many folks do in PyTorch is write that code --
a "training loop" to iterate over their `DataLoader`,
which in pseudocode might look something like:

```python
for batch in dataloader:
    inputs, targets = batch

    outputs = model(inputs)
    loss = some_loss_function(targets, outputs)
    
    optimizer.zero_gradients()
    loss.backward()

    optimizer.step()
```

PyTorch Lightning is a popular framework on top of PyTorch.

At its core, PyTorch Lightning provides

1. the `pl.Trainer` class, which organizes and executes your training, validation, and test loops, and
2. the `pl.LightningModule` class, which links optimizers to models and defines how the model behaves during training, validation, and testing.

Both of these are kitted out with all the features
a cutting-edge deep learning codebase needs:
- flags for switching device types and distributed computing strategy
- saving, checkpointing, and resumption
- calculation and logging of metrics

and much more.

Importantly these features can be easily
added, removed, extended, or bypassed
as desired, meaning your code isn't constrained by the framework.

#The `pl.LightingModule`

The first of our two core classes, the LightningModule, is like a souped-up torch.nn.Module -- it inherits all of the Module features, but adds more.

A LightningModule organizes your PyTorch code into 6 sections:


*   Initialization
*   Train Loop
*   Validation Loop
*   Test Loop
*   Prediction Loop
*   Optimizers and LR Schedulers

In [7]:
import pytorch_lightning as pl
from text_recognizer.lit_models import BaseLitModel

A `pl.LightningModule` is a `torch.nn.Module`,
so the basic definition looks the same:
we need `__init__` and `forward`.

In [8]:
class LinearRegression(pl.LightningModule):

  def __init__(self):
    super().__init__()

    self.model = torch.nn.Linear(in_features = 1, out_features = 1)

  def forward(self, xs):
    return self.model(xs)

In [12]:
import logging
import textwrap
import traceback

try:
  logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

  model = LinearRegression()

  trainer = pl.Trainer(accelerator="gpu", max_epochs=1)
  trainer.fit(model = model)

except pl.utilities.exceptions.MisconfigurationException as error:
  print("Error", *textwrap.wrap(str(error), 80), sep="\n\t")

finally:
  logging.getLogger("pytorch_lightning").setLevel(logging.INFO)

Error
	No `training_step()` method defined. Lightning `Trainer` expects as minimum a
	`training_step()`, `train_dataloader()` and `configure_optimizers()` to be
	defined.


The error message says we need some more methods.

Two of them are mandatory components of the `LightningModule`: `.training_step` and `.configure_optimizers`.

##.training_step

This method tells what to do during a single training step

In [ ]:
from typing import Tuple

def training_step(self: pl.LightningModule, batch: Tuple[torch.tensor, torch.Tensor], batch_idx: int) -> torch.Tensor:
  xs, ys = batch
  outs = self(xs)
  loss = torch.nn.functional.mse_loss(outs, ys)
  return loss

LinearRegression.training_step = training_step

##.configure_optimizers

#The pl.Trainer

#Training with PyTorch Lightning in the FSDL Codebase

#Extra Goodies

##pl.LightingDataModule

##pl.Callback

##torchmetrics